In [56]:
import altair as alt
import polars as pl
import duckdb


In [57]:
with duckdb.connect("pika_jobs.duckdb") as conn:
    query = """
    select DISTINCT metric, unit from stats;
    """

    units = conn.sql(query).pl()

units

metric,unit
str,str
"""mem_used""","""B"""
"""flops_any""","""F/s"""
"""mem_bw""","""B/s"""
"""net_bw""","""B/s"""
"""ipc""","""IPC"""
"""cpu_used""","""load"""
"""cpu_power""","""W"""


In [68]:
# Plot cpu usage of the jobs

with duckdb.connect("pika_jobs.duckdb") as conn:

    query = """
    select avg(value) as avg_value, metric, count(distinct date_time) * 30 as duration from stats group by job_id, metric;
    """

    averages_df = conn.sql(query).pl()

    # averages_df = averages_df.group_by("metric").agg(pl.col("avg_value").hist(include_breakpoint=True, include_category=True, bin_count=10))


    # query = """
    # select metric, histogram(avg_value) as hist
    # from (select avg(value) as avg_value, metric from stats group by job_id, metric) group by metric;
    # """

    # metrics_histogram = conn.sql(query).pl()


# metrics_histogram = metrics_histogram.explode("hist") \
#     .with_columns(pl.col("hist").struct.unnest()) \
#     .rename({
#         "key": "bucket",
#         "value": "count"
#     })

# metrics_hist = averages_df.explode("avg_value").with_columns(pl.col("avg_value").struct.unnest())



alt.Chart(averages_df.filter(pl.col("metric").eq("cpu_used")), title="CPU usage distribution").mark_bar().encode(
    x=alt.X("avg_value", title="average CPU usage over job lifetime").bin().axis(format=".0%"),
    y = alt.Y("count()", title="Count"),
).display(scaleFactor=2)


alt.Chart(...)

In [59]:
with duckdb.connect("pika_jobs.duckdb") as conn:

    query = """
    select job_id, sum(avg_value) as network_avg_usage from (
     select avg(value) as avg_value, job_id from stats where metric = 'net_bw' group by job_id, hostname
     ) group by job_id;
    """

    network_usage_df = conn.sql(query).pl()

alt.Chart(network_usage_df, title="Distribution of Infiniband network usage").mark_bar().encode(
    x=alt.X("network_avg_usage", title="average Infiniband network usage over job lifetime").bin(maxbins=11).axis(format=".2s"),
    y = alt.Y("count()", title="Count"),
).display(scaleFactor=2)

alt.Chart(...)

In [62]:
alt.Chart(averages_df.filter(pl.col("metric").eq("mem_bw")), title="mem read write").mark_bar().encode(
    x=alt.X("avg_value", title="average CPU usage over job lifetime").bin().axis(format=".2s"),
    y = alt.Y("count()", title="Count"),
).display(scaleFactor=2)

alt.Chart(...)

In [61]:
averages_df.filter(pl.col("metric").eq("mem_used"))

avg_value,metric
f64,str
2.0991e11,"""mem_used"""
2.0182e11,"""mem_used"""
2.0229e11,"""mem_used"""
2.0499e11,"""mem_used"""
1.9791e11,"""mem_used"""
…,…
5.4338e10,"""mem_used"""
1.4224e10,"""mem_used"""
1.9449e10,"""mem_used"""
